<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/swisslog_semantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.4 MB/s eta 0:00:00


In [2]:
from transformers import BertTokenizer, BertModel
import torch


[link text](https://github.com/google-research/bert#pre-trained-models)

Pre-trained models

We are releasing the BERT-Base and BERT-Large models from the paper. Uncased means that the text has been lowercased before WordPiece tokenization, e.g., John Smith becomes john smith. The Uncased model also strips out any accent markers. Cased means that the true case and accent markers are preserved. Typically, the Uncased model is better unless you know that case information is important for your task (e.g., Named Entity Recognition or Part-of-Speech tagging).

These models are all released under the same license as the source code (Apache 2.0).

For information about the Multilingual and Chinese model, see the Multilingual README.

When using a cased model, make sure to pass --do_lower=False to the training scripts. (Or pass do_lower_case=False directly to FullTokenizer if you're using your own script.)

The links to the models are here (right-click, 'Save link as...' on the name):

BERT-Large, Uncased (Whole Word Masking): 24-layer, 1024-hidden, 16-heads, 340M parameters

BERT-Large, Cased (Whole Word Masking): 24-layer, 1024-hidden, 16-heads, 340M parameters

BERT-Base, Uncased: 12-layer, 768-hidden, 12-heads, 110M parameters

BERT-Large, Uncased: 24-layer, 1024-hidden, 16-heads, 340M parameters

BERT-Base, Cased: 12-layer, 768-hidden, 12-heads , 110M parameters

BERT-Large, Cased: 24-layer, 1024-hidden, 16-heads, 340M parameters

BERT-Base, Multilingual Cased (New, recommended): 104 languages, 12-layer, 768-hidden, 12-heads, 110M parameters

BERT-Base, Multilingual Uncased (Orig, not recommended) (Not recommended, use Multilingual Cased instead): 102 languages, 12-layer, 768-hidden, 12-heads, 110M parameters

In [3]:
#Uncased means that the text has been lowercased before WordPiece tokenization
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)


In [4]:
sentences = ['Another sentence goes here.']

# Tokenize and convert to tensor
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

# Forward pass through the model
with torch.no_grad():
    outputs = model(**inputs)




In [5]:
print(inputs)

{'input_ids': tensor([[ 101, 2178, 6251, 3632, 2182, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


The outputs variable will contain a dictionary with several keys, including:

last_hidden_state: A tensor of shape (batch_size, sequence_length, hidden_size), containing the contextual embeddings for each token in the input sentence. In this case, since there is only one sentence, the tensor will have shape (1, sequence_length, hidden_size).

pooler_output: A tensor of shape (batch_size, hidden_size), representing a pooled representation of the entire input sentence. This tensor is obtained by applying a pooling operation (typically mean or max pooling) to the contextual embeddings of all tokens in the sentence.

hidden_states: A tuple of tensors containing the hidden states of the model at various layers. The number of hidden states will depend on the architecture of the BERT model.

In [7]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [8]:
outputs.last_hidden_state.shape   #outputs['last_hidden_state'].shape

torch.Size([1, 7, 768])

In [11]:
outputs.last_hidden_state.mean(dim=1).shape # mix all words of the sentence

torch.Size([1, 768])

In [10]:
outputs.pooler_output.shape

torch.Size([1, 768])

In [12]:
a = ['Receiving', 'block', '<*>' ]
sen = ' '.join(a)
sen

'Receiving block <*>'

In [1]:
from transformers import BertTokenizer, BertModel
import torch

def vectorize_sentences(sentences, model_name='bert-base-uncased'):
    # Load pre-trained BERT model and tokenizer
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    # Tokenize and convert to tensors for all sentences
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

    # Forward pass through the model
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract sentence embeddings (mean of token embeddings)
    sentence_embeddings = outputs.last_hidden_state.mean(dim=1)

    return sentence_embeddings


In [2]:
sentences_to_vectorize = [
    "This is the first sentence.",
    "Another sentence goes here.",
    "And a third sentence for vectorization."
]

# Get the vector representations of the sentences
vector_representations = vectorize_sentences(sentences_to_vectorize)

# Print the vector representations
print(vector_representations)

tensor([[-0.2604, -0.2182,  0.2819,  ...,  0.0782,  0.1706,  0.0177],
        [ 0.3117,  0.0221,  0.1525,  ...,  0.0644,  0.0831,  0.2578],
        [-0.1589, -0.2530, -0.2959,  ..., -0.0673, -0.1896,  0.2215]])


In [3]:
vector_representations.shape

torch.Size([3, 768])

In [4]:
from tqdm import tqdm
import pickle

t2wPath = '/content/templates.pkl'
outputPath = '/content/bert_encoding.pkl'

with open(t2wPath, 'rb') as f:
    data = pickle.load(f)
# group all words into one sentence
templateSentence = dict()
for i, v in tqdm(data.items()):
    sen = ' '.join(v)  #This line joins the list of words for each template into a single sentence, separated by spaces
    templateSentence[i] = vectorize_sentences(sen)

with open(outputPath, 'wb') as f:
    pickle.dump(templateSentence,f)

print('Successfully Finished BERT Encoding')

100%|██████████| 35/35 [01:25<00:00,  2.46s/it]

Successfully Finished BERT Encoding


In [6]:
templateSentence[0].shape

torch.Size([1, 768])

In [7]:
templateSentence.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34])